In [1]:
import torch
import utils
import os
import numpy as np
from collections import OrderedDict
from vgg_face_dag import vgg_face_dag
from ami_model import AmIModel

device = torch.device('cuda')
# vgg_weight = './vgg_face_caffe.pth'
vgg_weight = './vgg_face_dag.pth'
# vgg_weight = './keras_vgg_face.pth'
vgg_net = vgg_face_dag(vgg_weight)

In [2]:
vgg_net.to(device)
vgg_net.eval()

SKIP_LAYERS = utils.SKIP_LAYERS
ami_model = AmIModel(vgg_net)
ami_model.eval()
ami_model.register_my_hook(skip_layers=SKIP_LAYERS)

# ami_model.show_layers()

register hook for relu1_1
register hook for relu1_2
register hook for pool1
register hook for relu2_1
register hook for relu2_2
register hook for pool2
register hook for relu3_1
register hook for relu3_2
register hook for relu3_3
register hook for pool3
register hook for relu4_1
register hook for relu4_2
register hook for relu4_3
register hook for pool4
register hook for relu5_1
register hook for relu5_2
register hook for relu5_3
register hook for pool5
register hook for relu6
register hook for relu7


In [3]:
attributes = ['leye', 'reye', 'nose', 'mouth']
actions    = ['substitution', 'preservation']
root_path  = '../../data/attribute_mutated/'
base_img = '../../data/base_img.jpg'

base_img_tensor = utils.get_data(base_img).to(device)

In [4]:
def extract_witness(model, attri, base_img_tensor):
    neuron_set_lists = OrderedDict()
    model(base_img_tensor)
    org_layouts = model.get_activation_values()
    
    for n,l in org_layouts.items():
        neuron_set_lists[n] = set(range(l.shape[0]))
    for img_name in os.listdir(root_path + attri + '_' + actions[0]):
        attri_sub = root_path + attri + '_' + actions[0] + '/' + img_name
        attri_pre = root_path + attri + '_' + actions[1] + '/' + img_name
        
        attri_sub_img_tensor = utils.get_data(attri_sub).to(device)
        attri_pre_img_tensor = utils.get_data(attri_pre).to(device)
        
        model(attri_sub_img_tensor)
        attri_sub_layouts = model.get_activation_values()
        assert len(org_layouts) == len(attri_sub_layouts)
        
        for layer_idx in org_layouts.keys():
            # print(f'strengthen computing {layer_idx}')
            if attri_sub_layouts[layer_idx].shape != org_layouts[layer_idx].shape:
                print(f'{layer_idx} has different shape, {attri_sub_layouts[layer_idx].shape} != {org_layouts[layer_idx].shape}')
            if len(org_layouts[layer_idx].shape) == 1:
                # fc layers
                diff_layout = np.abs(attri_sub_layouts[layer_idx] - org_layouts[layer_idx])
            else:
                diff_layout = np.sum(np.abs(attri_sub_layouts[layer_idx] - org_layouts[layer_idx]), axis=(1,2))
            sub_set = set([i for i,v in enumerate(diff_layout) if v > np.median(diff_layout)])
            neuron_set_lists[layer_idx].intersection_update(sub_set) 
        
        model(attri_pre_img_tensor)
        attri_pre_layouts = model.get_activation_values()
        assert len(org_layouts) == len(attri_pre_layouts)
        
        for layer_idx in org_layouts.keys():
            # print(f'weaken computing {layer_idx}')
            if len(org_layouts[layer_idx].shape) == 1:
                # fc layers
                diff_layout = np.abs(attri_pre_layouts[layer_idx] - org_layouts[layer_idx])
            else:
                diff_layout = np.sum(np.abs(attri_pre_layouts[layer_idx] - org_layouts[layer_idx]), axis=(1,2))
            sub_set = set([i for i,v in enumerate(diff_layout) if v < np.median(diff_layout)])
            neuron_set_lists[layer_idx].intersection_update(sub_set)
    
    res = []
    for name, neuron_set in neuron_set_lists.items():
        res.append('%s->%s' % (name, ','.join(map(str,sorted(neuron_set)))))
        
    print('Extracted witnesses:\n\t%s' % ('\n\t'.join(res)))
    
    with open(os.path.join('ami_data', f'{attri}_neurons.txt'), 'w') as out_file:
        out_file.write('\n'.join(res))

In [5]:
for attri in attributes:
    print(f'Extracting witnesses for {attri}')
    extract_witness(ami_model, attri, base_img_tensor)

Extracting witnesses for leye
Extracted witnesses:
	relu1_1->42
	relu1_2->
	pool1->
	relu2_1->
	relu2_2->28,102
	pool2->28,102,106
	relu3_1->9,150,181,199
	relu3_2->20,126
	relu3_3->1,114,173
	pool3->1,181
	relu4_1->33,192,209,259,267,323,378,410,456
	relu4_2->181,206,292,392,481
	relu4_3->25,45,75,98,148,171,181,187,245,251,346,355,468,480,493
	pool4->75,91,171,187,329,468,484
	relu5_1->4,43,50,56,73,157,236,296,306,317,388,468
	relu5_2->147,223,328,377
	relu5_3->387
	pool5->227
	relu6->
	relu7->
Extracting witnesses for reye
Extracted witnesses:
	relu1_1->42
	relu1_2->
	pool1->
	relu2_1->
	relu2_2->28,69,102
	pool2->28,102,106
	relu3_1->6,9,150,181
	relu3_2->126,141,223
	relu3_3->98,236
	pool3->11,98,236
	relu4_1->26,33,135,192,267,323,410
	relu4_2->74,263,392
	relu4_3->0,25,91,108,252,273,274,355,446,468
	pool4->25,91,108,171,274,329,355,468,475
	relu5_1->87,157,177,249,324,384,416,468,483
	relu5_2->232
	relu5_3->
	pool5->
	relu6->
	relu7->
Extracting witnesses for nose
Extracted wi